In [2]:
import pickle as pkl
import scipy.stats as stats
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
%matplotlib inline

In [3]:
with open('CL_clean_car_all.pkl', 'rb') as picklefile: 
    df1 = pkl.load(picklefile)
    
del df1['Vin']
del df1['Cylinders']
del df1['Paint Color']
del df1['Make Model']
del df1['Diff']

In [4]:
df1[df1['Year']<=1950]

,Condition,Drive,Fuel,Odometer,Price,Size,Title Status,Transmission,Type,Year,Make,Model
36,good,rwd,gas,NaN,3200.0,NaN,clean,manual,other,1948,Ford,F5 Truck
262,NaN,NaN,gas,NaN,29500.0,NaN,clean,manual,NaN,1928,Ford,Model T
270,NaN,NaN,gas,NaN,4500.0,NaN,clean,automatic,NaN,1950,Chevrolet,
309,like new,rwd,gas,1200,37500.0,NaN,clean,automatic,convertible,1932,Ford,
431,NaN,NaN,gas,NaN,8500.0,NaN,clean,manual,NaN,1940,None,
461,good,rwd,gas,61000,9500.0,NaN,clean,manual,sedan,1928,None,
562,NaN,NaN,gas,NaN,42000.0,NaN,clean,automatic,NaN,1932,Ford,
586,excellent,NaN,gas,NaN,40000.0,NaN,clean,automatic,NaN,1929,Ford,
598,excellent,NaN,gas,NaN,40000.0,NaN,clean,automatic,NaN,1929,Ford,
736,good,4wd,gas,NaN,1800.0,NaN,clean,manual,NaN,1949,Jeep,


In [28]:
import numpy as np
def model_nan(string):
    if string=='':
        return(np.nan)
    else:
        return(string)
df1['Model']=df1['Model'].apply(model_nan)

In [ ]:
import missingno as msno
msno.matrix(df1.head(4779))

In [88]:
del df1['Drive']
del df1['Vin']
del df1['Cylinders']
del df1['Diff']
del df1['Paint Color']
del df1['Make Model']
del df1['Size']
del df1['Condition']
del df1['Type'] # about 9 categorical variables, and so this can be worked with later

In [89]:
# converting all odometer to int type
import numpy as np
def odo_str_to_int(string):
    if string==(np.nan):
        return(np.nan)
    else:
        if type(string)==int:
            return(string)
        elif type(string)==float:
            return(string)
        elif type(string)==str:
            if string.isnumeric():
                return(int(string))
        else:
            pass

In [90]:
df1['Odometer']=df1['Odometer'].apply(odo_str_to_int)

In [91]:
df1.sample(50)

,Fuel,Odometer,Price,Title Status,Transmission,Year,Make,Model
2314,gas,NaN,850.0,clean,automatic,2000,Mercedes-Benz,
1972,gas,135000.0,3000.0,clean,automatic,1999,Ram,
2519,gas,151816.0,5995.0,clean,automatic,2006,Chrysler,300
2132,gas,145000.0,2000.0,clean,automatic,1991,Lexus,LS Hybrid
2070,gas,70000.0,6000.0,clean,manual,1995,Toyota,
282,gas,160.0,2800.0,clean,automatic,2002,Ford,Explorer Sport
4056,gas,28319.0,NaN,clean,automatic,2016,GMC,Suburban 1500
4272,other,NaN,NaN,clean,automatic,2013,Honda,Pilot
196,gas,160000.0,800.0,rebuilt,manual,1998,Volkswagen,Jetta
1101,gas,165.0,1900.0,clean,manual,1998,Ford,ZX2


In [92]:
df1['Fuel'].value_counts()

gas         4106
other        295
diesel       291
hybrid        58
electric      27
Name: Fuel, dtype: int64

In [93]:
df1['Title Status'].value_counts()

clean         4440
rebuilt        208
salvage         74
missing         27
lien            16
parts only      14
Name: Title Status, dtype: int64

In [94]:
df1['Transmission'].value_counts()

automatic    3802
manual        800
other         154
               21
Name: Transmission, dtype: int64

In [95]:
df1=df1.dropna(how='any')
df1=df1[df1['Model']!='']
df1.sample(50)

,Fuel,Odometer,Price,Title Status,Transmission,Year,Make,Model
1216,gas,227000.0,3000.0,clean,automatic,1993,Toyota,Previa
3713,gas,72193.0,12999.0,clean,manual,1998,BMW,8 Series
2696,gas,16262.0,39838.0,clean,,2015,Toyota,4Runner
3789,gas,20102.0,30999.0,clean,automatic,2016,Toyota,Truck Xtracab SR5
2126,gas,100000.0,12000.0,clean,automatic,2013,Mazda,Mazdaspeed 3
3519,gas,123586.0,6999.0,clean,automatic,2005,Volvo,XC90
3814,gas,107417.0,18444.0,clean,automatic,2007,Cadillac,Escalade
1143,gas,108000.0,8700.0,clean,automatic,2002,BMW,3 Series
2413,gas,19685.0,27900.0,clean,automatic,2015,Chevrolet,Cobalt SS
1218,gas,15000.0,30000.0,clean,automatic,2015,Honda,CR-V


In [96]:
with open('CL_super_clean_car_all.pkl', 'wb') as picklefile:
    pkl.dump(df1, picklefile)